In [238]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
import random
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.multiclass import OutputCodeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.pipeline import Pipeline as ImPineline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.grid_search import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import lfilter
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

import time
%matplotlib inline

In [239]:
# spec = total_train_feature.iloc[:,:-1].head()

def specLineNum(spec, windowSize=2, threshold=0.8):
    fs = pd.Series()
    ss = pd.Series()
    for i in range(windowSize, spec.shape[1]):
        s = spec.iloc[:,i-windowSize]
        e = spec.iloc[:,i]
        isFs = (e>threshold) & ((e-s)>threshold)
        isSS = (e<threshold) & ((e-s)<-threshold)
        if(fs.size==0):
            fs = isFs.astype(int)
        else:
            fs = fs + isFs.astype(int)
        if(ss.size==0):
            ss = isSS.astype(int)
        else:
            ss = ss + isSS.astype(int)
    combine = pd.concat([fs, ss], axis=1)
    combine.columns = ['fs','ss']
    return combine

In [240]:
def qsoFeature(data):
    std1 = data.iloc[:,850:880].std(axis=1)
    std2 = data.iloc[:,885:898].std(axis=1)
    std3 = data.iloc[:,900:1000].std(axis=1)
    f1 = (std1<0.3) & (std2>0.35) & (std3<0.35)
    f3 = (std2*std2)/(std1*std3+0.1)
    d1 = std1+0.2
    f2 = (std2>d1)
    qsof = pd.concat([f1, f2, f3], axis=1)
    qsof.columns = ['qso_f1','qso_f2','qso_f3']
    return (qsof)

In [241]:
def intervalEWFeature(d, start=0, end=-1, filterThrehold=1):
    cp = d
#     cp[abs(cp)<filterThrehold] = 0
    rd = cp.iloc[:, start:end]
    specLines = specLineNum(rd[abs(rd)>filterThrehold])
    ew_absorb = rd[(rd<0)&(abs(rd)>filterThrehold)].sum(axis=1)
    ew_fs = rd[(rd>0)&(abs(rd)>filterThrehold)].sum(axis=1)
    return pd.concat([ew_absorb, ew_fs, specLines], axis=1)

def smooth(filterSet):
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filterSet = lfilter(b, a, filterSet, axis=1)
    return pd.DataFrame(filterSet)

def EWByRangeList(d, rangeList, filterThrehold=1, isSmooth=False, doScale=True):
    if doScale:
        d = pd.DataFrame(scale(d, axis=1))
    if isSmooth:
        d=smooth(d)
    ewfAll = pd.DataFrame()
    for r in rangeList:
        start, end = r
        ewf = intervalEWFeature(d, start, end, filterThrehold)
        newNames = ['ew_' + str(start) + '_' + str(end) + '_' + str(v) for v in ewf.columns.values]
        ewf.columns = newNames
        ewfAll = pd.concat([ewfAll,ewf], axis=1)
    return ewfAll

In [242]:
def submit(fileName, testTotal, preds):
    submission = testTotal[['id']]
    submission['pred'] = preds
    submission.to_csv('./predicts/'+fileName, header=False, index=False)
    return submission
    

In [243]:
def extractFeature(train_feature, partition=2, randomPartion=False, prefix="p", onlyAvg=False, 
                   filterLow=False, levelFeature=False, scaleSpec=False, smooth=False,
                   generalFeature = True,
                  ):
    if scaleSpec:
        train_feature = pd.DataFrame(scale(train_feature, axis=1))
    if smooth:
        train_feature = train_feature.rolling(100, axis=1).mean()
    if filterLow:
        train_feature[train_feature<1] = 0
    plen = train_feature.shape[1]/partition
    features = pd.DataFrame()
    avgs = pd.DataFrame()
    for i in range(0, partition):
        pstart = random.randint(0, train_feature.shape[1]-plen-1)
        pendExclue = pstart + plen + 1
        avgC = train_feature.iloc[:, pstart:pendExclue].mean(axis=1)
        stdC = train_feature.iloc[:, pstart:pendExclue].std(axis=1)
        maxC = train_feature.iloc[:, pstart:pendExclue].max(axis=1)
        minC = train_feature.iloc[:, pstart:pendExclue].min(axis=1)
        medianC = train_feature.iloc[:, pstart:pendExclue].median(axis=1)
        diffC = train_feature.iloc[:, pstart:pendExclue].diff(axis=1).iloc[:,1:]
        features[prefix+'_avg'+str(i)] = avgC
        avgs[prefix+'_avg'+str(i)] = avgC
        if onlyAvg==False:
            features[prefix+'_std'+str(i)] = stdC
            features[prefix+'_median'+str(i)] = medianC
            features[prefix+'_max'+str(i)] = maxC
            features[prefix+'_min'+str(i)] = minC
            features[prefix+'_avg_m_std'+str(i)] = stdC/avgC
            features[prefix+'_max_m_avg'+str(i)] = maxC/avgC
            features[prefix+'_max_m_median'+str(i)] = maxC/(medianC+0.1)
#             features[prefix+'_median_m_mean'+str(i)] = medianC/avgC
#             features[prefix+'_min_m_median'+str(i)] = minC/(medianC+0.1)
            features[prefix+'_min_m_avg'+str(i)] = minC/avgC
#             features[prefix+'_media_m_avg'+str(i)] = medianC/avgC
    if levelFeature:
        level_feature = extractFeature(avgs, prefix="l", partition = 4)
        features = pd.concat([features, level_feature], axis = 1)
    if generalFeature:
        avgC = train_feature.mean(axis=1)
        stdC = train_feature.std(axis=1)
        maxC = train_feature.max(axis=1)
        minC = train_feature.min(axis=1)
        medianC = train_feature.median(axis=1)
        features[prefix+'_std'] = stdC
        features[prefix+'_median'] = medianC
        features[prefix+'_max'] = maxC
        features[prefix+'_min'] = minC
        features[prefix+'_avg_m_std'] = stdC/avgC
        features[prefix+'_max_m_avg'] = maxC/avgC
        features[prefix+'_max_m_median'] = maxC/(medianC+0.1)
        features[prefix+'_median_m_mean'] = medianC/avgC
        features[prefix+'_min_m_median'] = minC/(medianC+0.1)
        features[prefix+'_min_m_avg'] = minC/avgC
    return features.fillna(0)

In [244]:
def readTotalFeature(fileName, maxFileIndex, hasLabel=True, partition = 20, ewFeature=True, addQsoFeature=False, randEWNum=100):
    total_train_feature = pd.DataFrame()
    rangeList = [(0,300), (0, 500), (500, 1000),(850, 1000), (1000, 1300),(1500,2000), (1500, 2600), (2500, 2600)]
    for i in range(0, maxFileIndex+1):
        print('processing:' + str(i))
        train_set = pd.read_csv(fileName + str(i) +'.csv', header=None)
        renameCol = {0:'id'}
        if hasLabel:
            renameCol[train_set.shape[1]-1] = 'label'
        train_set.rename(columns=renameCol, inplace=True)
        if hasLabel:
            processed_train = pd.DataFrame()
            rawFeature = train_set.iloc[:,1:-1]
        else:
            rawFeature = train_set.iloc[:,1:]
        processed_train = extractFeature(rawFeature, partition, levelFeature=False, smooth=False)
        if ewFeature:
            ew_feature = EWByRangeList(rawFeature, rangeList, filterThrehold=0.5, doScale=True)
            processed_train = pd.concat([processed_train, ew_feature], axis=1)
        if addQsoFeature:
            qso_feature = qsoFeature(rawFeature)
            processed_train = pd.concat([processed_train, qso_feature], axis=1)
        if hasLabel:
            processed_train['label'] = train_set.label
        else:
            processed_train['id'] = train_set.id
        total_train_feature = total_train_feature.append(processed_train, ignore_index=True)
    return total_train_feature.sample(total_train_feature.shape[0], replace=False)

In [245]:
validMode = False
reFeature = True
partition = 45
rng = np.random.RandomState(42)

In [246]:
if reFeature:
    total_train_feature = readTotalFeature('./train_feature_all_', 48, partition=partition)
    total_train_feature.to_csv('./train_feature_check_point.csv', index=False)
else :
    total_train_feature = pd.read_csv('./train_feature_check_point.csv')

processing:0
processing:1
processing:2
processing:3
processing:4
processing:5
processing:6
processing:7
processing:8
processing:9
processing:10
processing:11
processing:12
processing:13
processing:14
processing:15
processing:16
processing:17
processing:18
processing:19
processing:20
processing:21
processing:22
processing:23
processing:24
processing:25
processing:26
processing:27
processing:28
processing:29
processing:30
processing:31
processing:32
processing:33
processing:34
processing:35
processing:36
processing:37
processing:38
processing:39
processing:40
processing:41
processing:42
processing:43
processing:44
processing:45
processing:46
processing:47
processing:48


In [247]:
total_train_feature = total_train_feature.fillna(0)

In [248]:
total_train_feature[total_train_feature.label=='unknown'].shape

(34288, 448)

In [249]:
X = total_train_feature.iloc[:,:-1]
Y = total_train_feature.iloc[:,-1]

# Split the dataset in two equal parts
x, t_x, y, t_y = train_test_split(
    X, Y, test_size=0.3, random_state=0)
x.shape


(338695, 447)

In [250]:
def outlier_rejection(X, y):
    model = IsolationForest(n_jobs=-1, random_state=rng)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

In [251]:
smote = SMOTE()
anova_filter = SelectKBest(f_classif, k=(int)(x.shape[1]*0.9))
model = RandomForestClassifier(n_jobs=4, class_weight='balanced', max_depth=60, n_estimators=10, bootstrap=False, verbose=1,random_state=rng)
# model = AdaBoostClassifier(random_state=rng)
# model = GradientBoostingClassifier(random_state=rng,verbose=1)
# model = XGBClassifier()
# model = KNeighborsClassifier(n_neighbors = 3, n_jobs=-1)
scaler = StandardScaler()
scaler = Normalizer()
poly = PolynomialFeatures(2)
# scaler = RobustScaler()
pca = PCA(n_components=50, svd_solver='full')
ref = RFE(estimator=LogisticRegression(), n_features_to_select=50)
# model = GaussianNB()
clf = ImPineline([
#     ('feture_select', anova_filter),
#     ('pca',pca),
#     ('ref', ref),
    ('scaler',scaler),
    ('smote',smote),
    ('model', model)
])


In [252]:
if validMode:
#     rs = clf.fit(x, y)
    scores = cross_val_score(clf, x, y, cv=3, n_jobs=-1, verbose=1, scoring='f1_macro')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#     preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
#     print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  

In [253]:
if validMode:
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
#     preds_test = carbateQso(t_x, preds_test)
#     clf.score(t_x, t_y)
    print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [254]:
if validMode:
    clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced',  verbose=1 )
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
    print('f1: %.2f' % f1_score(t_y, preds_test, average='macro')) 
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [255]:
if validMode:
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()


In [256]:
if validMode:
    smote = SMOTE()
    parameters = {
        'max_depth':[60, 70, 80],  #70
        'max_features':[0.5,0.6,0.7], #0.6
    #     'min_samples_split':[2, 4, 8, 20], #2
    #     'min_samples_leaf':[1,4,8,20], #1
    #     'min_weight_fraction_leaf':[0,0.05,0.1,0.2], #0
        'max_leaf_nodes':[800, 1000, 1200], #1000
    #      'bootstrap':[True, False], #False
    #     'oob_score':[True, False], #False
    #     'class_weight':[None,'balanced'], #balanced
        'n_estimators':[10, 20], #10
    #     'criterion':['gini','entropy']
    }
    clf = GridSearchCV(RandomForestClassifier(n_jobs=4, max_depth=20), parameters, cv=5,
                           scoring='f1_macro', n_jobs=8, verbose=2)
    clf.fit(x, y)
    print(clf.best_params_)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [257]:
if reFeature:
    testTotal = readTotalFeature('./test_feature_all_', 9, hasLabel=False, partition=partition)
    testTotal.to_csv('./test_feature_checkpoint.csv', index=False)
    testTotal.head()
else :
    testTotal = pd.read_csv('./test_feature_checkpoint.csv')

processing:0
processing:1
processing:2
processing:3
processing:4
processing:5
processing:6
processing:7
processing:8
processing:9


In [258]:
testTotal.shape

(100000, 448)

In [259]:
clf.fit(X, Y)



[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed: 11.1min finished


Pipeline(memory=None,
     steps=[('scaler', Normalizer(copy=True, norm='l2')), ('smote', SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=None, ratio='auto', svm_estimator=None)), ('model', RandomForestClassifier(bootstrap=False, class_weight='balanced',
           ...random_state=<mtrand.RandomState object at 0x1c18d2c5a0>,
            verbose=1, warm_start=False))])

In [260]:
test_x = testTotal.iloc[:,:-1]
pred = clf.predict(test_x)
timestr = time.strftime("%m%d_%H%M")
rs = submit(timestr+'_rf_ew_e100_pca_40.csv',testTotal, pred)


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.2s finished
/Users/cql/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
rs.pred.value_counts()

In [ ]:
total_train_feature.head()

In [ ]:
total_train_feature[:,:-1]